In [9]:
%matplotlib inline

# bunch of required imports
from __future__ import division, print_function
import os, json
from glob import glob
import numpy as np
from matplotlib import pyplot as plt
import utils; reload(utils)
from utils import plots

np.set_printoptions(precision=4, linewidth=100)

In [12]:
# data is in the following directory layout
!tree -d data

data
└── dogscats
    ├── models
    ├── sample
    │   ├── train
    │   │   ├── cats
    │   │   └── dogs
    │   └── valid
    │       ├── cats
    │       └── dogs
    ├── test1
    │   └── unknown
    ├── train
    │   ├── cats
    │   └── dogs
    └── valid
        ├── cats
        └── dogs

17 directories


In [13]:
# using the sample dir locally
path = 'data/dogscats/sample/'